<a href="https://colab.research.google.com/github/vectice/vectice-examples/blob/master/Notebooks/Vanilla/Handwritten_Digit_Recognition_with%20_keras/Handwritten_Digit_Recognition_with_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handwritten digits recognition 



## What this is about?
The handwritten digit recognition is the ability of computers to recognize human handwritten digits. Handwritten digits are not perfectly written which makes it hard for machine to recognize them. Handwritten digits recognition could be a solution for several problems, such as those which use the image of a digit and recognizes the digit present in the image, for example.

## Install vectice 

Vectice provides a generic metadata layer that is potentially suitable for most data science workflows.

For this tutorial we will use the keras library for modeling and track experiments directly through our Python SDK to illustrate how to fine-tune exactly what you would like to track: metrics, etc. The same mechanisms would apply to R, Java or even more generic REST APIs to track metadata from any programming language and library.

Here is a link to the Python SDK Documentation, it's not final nor complete and it is updated as we go along. 
[Python SDK Documentation](https://doc.vectice.com/)

In [ ]:
!pip3 install -q vectice[github]

In [ ]:
!pip show vectice

## Install the necessary libraries for this project

In [ ]:
!pip3 install -q tensorflow
!pip3 install -q keras
!pip3 install -q pillow
!pip3 install -q numpy

## Import the libraries and the dataset

In [ ]:
# organize imports
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.datasets import mnist
from keras.utils import np_utils
from vectice import Vectice
from vectice.models import JobType
from vectice.entity.model import ModelType
# fix a random seed for reproducibility
np.random.seed(9)

## User inputs for the model

In [ ]:
# user inputs
nb_epoch = 25
num_classes = 10
batch_size = 128
train_size = 60000
test_size = 10000
v_length = 784



- **nb_epoch**: Number of epochs. An epoch is an arbitrary cutoff, generally defined as "one pass over the entire dataset", used to separate training into distinct phases, which is useful for logging and periodic evaluation.
- **num_classes**: Digits classes (from 0 to 9)
- **bacth_size**: It defines the number of samples that will be propagated through the network. For exampe, if we have 1000 training samples, with a batch_size of 100 the model will train on the first 100 training samples then, the second 100 training sample on so on to the end of the training set. 
- **train_size**: Number of images in the training set
- **test_size**: Number of images in the test set
- **v_length**: Dimension of flattened input image size i.e. if input image size is [28x28], then v_length = 784

## Process the data

In this tutorial we will use the MNIST keras dataset. The MNIST dataset is a low-complexity data collection of handwritten digits containing 70,000 28x28 black and white images representing the digits zero through nine. It's used to train and test various supervised machine learning algorithms.


In [ ]:
# split the mnist data into train and test
(trainData, trainLabels), (testData, testLabels) = mnist.load_data()
print("[INFO] train data shape: {}".format(trainData.shape))
print("[INFO] test data shape: {}".format(testData.shape))
print("[INFO] train samples: {}".format(trainData.shape[0]))
print("[INFO] test samples: {}".format(testData.shape[0]))

trainData contains the data from which our neural network (model) will learn to recognize images and trainLabels contains the labels (what's in the picture). testData will be used to check if the model can correctly recognize a digit that it hasn't sees before. To evaluate this we will use DataLabels which contains labels for the test set

### Reshape and normalize the data
**Reshaping**

If we process an image in a neural network, it expects a vector and not a two-dimensional array (unless we use a convolution). Therefore, before further processing, we should convert the training set to 60,000 x 784 (28 * 28). To change the shape of the data we will use the reshape function.

**Normalization**

The value of each pixel – indicating the gray level of a given pixel – should be between 0 (typically 0 is completely black) and 255. We divide our data by 255 to make sure that it's between 0 and 1.

In [ ]:
# reshape the dataset
trainData = trainData.reshape(train_size, v_length)
testData = testData.reshape(test_size, v_length)
trainData = trainData.astype("float32")
testData = testData.astype("float32")
trainData /= 255
testData /= 255

print("[INFO] train data shape: {}".format(trainData.shape))
print("[INFO] test data shape: {}".format(testData.shape))
print("[INFO] train samples: {}".format(trainData.shape[0]))
print("[INFO] test samples: {}".format(testData.shape[0]))

In the process, we want the neural network to compare two values and then calculate how far is it from the target. This cannot be done without transforming the target (labels) into so-called one-hot encoding. As a result, our “1” will be written in the following form:

[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

If we wanted to write the value 5 in this way, the vector would look like this:

[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]

This operation of converting a scalar to a vector is performed by the function to_categorical().



In [ ]:
# convert class vectors to binary class matrices --> one-hot encoding
mTrainLabels = np_utils.to_categorical(trainLabels, num_classes)
mTestLabels = np_utils.to_categorical(testLabels, num_classes)

## Connect to your Vectice workspace and your Vectice project

Here we are going to need an API token and a project token. An API token is used to secure requests between your existing tools and Vectice. You can create and manage those at the API Tokens tab in your workspace, and they impersonate you and your rights per workspace, so we strongly recommend you to avoid sharing them. A project token is used to target the project you're working on in the UI and can found (after creating a project) in the Project settings page, and anyone working on the project can see it and copy/paste it.

In [ ]:
# In order to use Vectice SDK, let's set up the configurations first.
# The Vectice API key below can be generated from the UI.
# For better security, the settings can also be put into a dedicated file called `.vectice` or `.env` if you're working locally.

## Here, we specify the Vectice API endpoint
os.environ['VECTICE_API_ENDPOINT']= "beta.vectice.com"

##Complete with your Vectice API token
os.environ['VECTICE_API_TOKEN'] = ""

## Complete with your Vectice project token
Project_Token = "PROJECT TOKEN"

## Create a Vetice instance to connect to your project using your project token
vectice = Vectice(project_token=Project_Token)

print(vectice)

Create a dataset version (empty in this case since we download our data directly from here https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz) containing your data to use them as input for your models. That can be done through the UI by going to your project, clicking on datasets and then clicking on add (you have to add a connection to be able to create a dataset (the service account (readerKey.json) used to create the connection must have writing rights to do that)).

We can also create a dataset from the SDK:
We can either create a dataset using the connection name, or the connection id. For both methods, we should specify the connection id/name, the dataset name, the list of files and the list of folders that our dataset will contain. To create a dataset containing just folders (no files) we should put None in the files list argument when we call one of the two functions. For exemple:

- **vectice.create_dataset_with_connection_id(Connection_id, "dataset_versioning", files)** and **vectice.create_dataset_with_connection_name("Connection_name", "dataset_versioning",files)** enable to create a dataset named dataset_versioning using the connection whose id/name is Connection_id/Connection_name and add the list of files files to the dataset.

- **vectice.create_dataset_with_connection_id(Connection_id, "dataset_versioning", files, folders)** and **vectice.create_dataset_with_connection_name("Connection_name", "dataset_versioning", files, folders)** enable to create a dataset named dataset_versioning using the connection whose id/name is Connection_id/Connection_name and add the list of files files and the list of folders folders to the dataset.
 
- **vectice.create_dataset_with_connection_id(Connection_id, "dataset_versioning", None, folders)** and **vectice.create_dataset_with_connection_name("Connection_name", "dataset_versioning", None, folders)** enable to create a dataset named dataset_versioning using the connection whose id/name is Connection_id/Connection_name and add the list of folders folders to the dataset.


We can check if the datasets are already created in our workspace by calling **vectice.list_datasets()** which lists all the datasets existing in the project

In [ ]:
vectice.list_datasets().list

Create a dataset version based on the created/existing dataset that contains your data

In [ ]:
##We use autoversioning here
input_ds_version = vectice.create_dataset_version().with_parent_name("dataset_name")

## Get different user versions
Generate a random user version by calling get_random_string

In [ ]:
# Let's generate some unique names for our following modeling experiments
import random
import string
def get_random_string(length):
    return "".join(random.choice(string.ascii_letters) for i in range(length))


## Model creation

We can get the list of the models existing in our project by calling **vectice.list_models()**

In [ ]:
vectice.list_models().list

We are using a [sequential model](https://keras.io/getting-started/sequential-model-guide/) (a Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor) and two types of layers: Dense and Dropout.
Dense is a standard layer of the neural network in which each neuron is connected to each neuron of the next layer. Dropout is a layer that prevents the phenomenon of over-fitting, i.e. over-matching the operation of the neural network to training data. 
- We are using 3 Dense layers and 2 Dropout layers.
-  Each layer has an activation function. Activation functions available in Keras are listed here: https://keras.io/activations/
- Each layer as the first argument takes the size of the output vector for the layer (the first layer accepts 784 parameters and outputs 512)
- We use Dropout in our neural network to save it from overfitting. To prevent overfitting, it randomly chooses a fraction of units and set to 0 at each update


In [ ]:
## Here, we create a code version in order to use it as input for our runs to show the location of the source code
## This notebook is in a Github repository, so we're going to use vectice.create_code_version_with_github_uri to create the code version
uri = "https://github.com/vectice/vectice-examples"
script_relative_path="Notebooks/Vanilla/Handwritten_Digit_Recognition_with%20_keras/Handwritten_Digit_Recognition_with_keras.ipynb"
input_code = Vectice.create_code_version_with_github_uri(uri=uri, script_relative_path=script_relative_path)

The following code creates a DNN (Dense Neural Network) model. Complete the code by creating and starting a job run.

In [ ]:
# create the model

vectice.create_run(job_name = "Train model with keras", job_type = JobType.TRAINING)

vectice.start_run(inputs=[input_ds_version, input_code])

model = Sequential(name="Digits_recognition_sequential")
model.add(Dense(512, input_shape=(784,)))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Dense(num_classes))
model.add(Activation("softmax"))

# summarize the model
model.summary()

Here we define a list containing information about each layer in our neural network

In [ ]:
layers_list = []
for layer in model.layers:
  layers_list+=[("Layer name", str(layer.name)+" layer"), (str(layer.name)+" layer "+" number of input parameters", str(layer.input_shape[1])),(str(layer.name)+" layer "+" number of output parameters",str(layer.output_shape[1]))]
layers_list

After creating the model, we have to compile it. To do this, we call the method compile specifying:
- Type of optimizer: https://keras.io/optimizers/
- Loss function: https://keras.io/losses/
- Optional metrics that we want for our model to register (accuracy for example)

In [ ]:
# compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

Implementing the training of the neural network by using the fit method.

You can try to play with the network – change the number of epochs, batch_size, type of optimizer or the structure of the network itself.

In [ ]:
# fit the model
history = model.fit(trainData, mTrainLabels, validation_data=(testData, mTestLabels), batch_size=batch_size, epochs=nb_epoch, verbose=2)

## Model evaluation

Here we calculate some metrics in order to evaluate the model.
We already created a job run above and started it. We complete it by creating a model version, adding the metrics and the parameters (properties) to it and end the run.

In [ ]:
# print the history keys
print(history.history.keys())
# evaluate the model
scores = model.evaluate(testData, mTestLabels, verbose=0)

# history plot for accuracy
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Model Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(["train", "test"], loc="upper left")
plt.show()

# history plot for accuracy
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Model Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(["train", "test"], loc="upper left")
plt.show()

# print the results
print("[INFO] test score - {}".format(scores[0]))
print("[INFO] test accuracy - {}".format(scores[1]))


metrics = [('test score',scores[0]), ("test accuracy",scores[1])]

properties = [("Model", "Sequential (keras)")] +layers_list+ [("Number of epochs",str(nb_epoch)), ("number of digit classes (from 0 to 9) ", str(num_classes)), ("Batch size ", str(batch_size)), ("Number of images in the training set ", str(train_size)), ("Number of images in the test set ", str(test_size)), ("Dimension of flattened input image size", str(v_length))]
model_version1 = vectice.create_model_version().with_parent_name("Classifier").with_algorithm("Neural network").with_type(ModelType.CLASSIFICATION).with_properties(properties).with_metrics(metrics).with_user_version(get_random_string(12))


vectice.end_run(outputs=[model_version1])

The model's got an accuracy of 0.9826 on the test set. The difference between accuracy on the training and test set is only 0.01% and seems great

We can use our testData to test the model on picture it hasn't seen before. We can do so by using model.predection in keras

In [ ]:
predictions = model.predict(testData[0:100])

The method returns a 100-element scoreboard. Each element indicates the probabilities that the input belongs to a given class.

In [ ]:
predictions[0]

As we can see in this example the probabilities are very low except for the eight element (class 7, counted from 0) wich has a probability of 1. So the model predicted that it's 7. Let's verify that

In [ ]:
np.argmax(predictions[0])

In [ ]:
plt.imshow(testData[0].reshape(28,28))

As you can see the number is 7

We can take a look at the predictions (after treating the probabilities with the argmax function and specifying to the function in what dimension it should analyze data (in our case, along the y axis, i.e. axis = 1) because we are using a two-dimensional array)

In [ ]:
np.argmax(predictions, axis=1)

We can check the labels for the data on which the model did the predictions and compare

In [ ]:
testLabels[0:100]

In [ ]:
np.argmax(predictions, axis=1) == testLabels[0:100]

In [ ]:
np.mean(np.argmax(predictions, axis=1) == testLabels[0:100])


The predictions coincide with reality

We can also verify if there are wrong predictions by running this cell which uses Numpy's argmin function

In [ ]:
wrong_pred = np.argmin(np.argmax(predictions, axis=1) == testLabels[0:100])
wrong_pred

No element was wrongly predicted (for example, if wrong_pred=2 we can say that the the element in the position 2 is wrongly predicted)